In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import catboost
import lightgbm
import xgboost
%pylab inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestRegressor

Populating the interactive namespace from numpy and matplotlib


In [3]:
data_train = pd.read_csv('data/data_train_v6_1.csv')
data_test = pd.read_csv('data/data_test_v6_1.csv')
target = data_train.skilled

In [3]:
data_train['gold/gold_spent'] = (data_train['gold'] + 0.01) / (data_train['gold_spent'] + 0.01)
data_train['gold_spent/goldpermin'] = (data_train['gold_spent'] + 0.01) / (data_train['gold_per_min'] + 0.01)
data_train['gold_spent/net_worth'] = (data_train['gold_spent'] + 0.01) / (data_train['net_worth'] + 0.01)
data_train['is_player_team_win'] = (data_train['player_team'] == data_train['winner_team']).astype('int')
data_train['duration*gpm'] = (data_train['duration'] / 60) * data_train['gold_per_min']
data_train['duration*xpm'] = (data_train['duration'] / 60) * data_train['xp_per_min']
data_train['armor'] = data_train['base_armor'] + data_train['level'] * data_train['agi_gain'] / 6 
data_train['rate_range_attack'] = data_train['attack_range'] * data_train['attack_rate']

for i in ['skilled','gold_per_min','xp_per_min', 'avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_train.groupby('hero_id')[i].mean()
    data_train['gp_h{}_mean'.format(i)] = data_train['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_train.groupby('hero_id')[i].median()
    data_train['gp_h{}_median'.format(i)] = data_train['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_train.groupby('hero_id')[i].max()
    data_train['gp_h{}_max'.format(i)] = data_train['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_train['delta_{}_mean'.format(i)] = data_train['gp_h{}_mean'.format(i)] - data_train[i]
    
for i in ['gold_per_min','xp_per_min', 'avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_train['delta_{}_max'.format(i)] = data_train['gp_h{}_max'.format(i)] - data_train[i]

for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_train['delta_{}_median'.format(i)] = data_train['gp_h{}_median'.format(i)] - data_train[i]

In [4]:
data_test['gold/gold_spent'] = (data_test['gold'] + 0.01) / (data_test['gold_spent'] + 0.01)
data_test['gold_spent/goldpermin'] = (data_test['gold_spent'] + 0.01) / (data_test['gold_per_min'] + 0.01)
data_test['gold_spent/net_worth'] = (data_test['gold_spent'] + 0.01) / (data_test['net_worth'] + 0.01)
data_test['is_player_team_win'] = (data_test['player_team'] == data_test['winner_team']).astype('int')
data_test['duration*gpm'] = (data_test['duration'] / 60) * data_test['gold_per_min']
data_test['duration*xpm'] = (data_test['duration'] / 60) * data_test['xp_per_min']
data_test['armor'] = data_test['base_armor'] + data_test['level'] * data_test['agi_gain'] / 6 
data_test['rate_range_attack'] = data_test['attack_range'] * data_test['attack_rate']

mn_1 =data_train.groupby('hero_id')['skilled'].mean()
data_test['gp_hskilled_mean'] = data_test['hero_id'].map(mn_1)

for i in ['gold_per_min','xp_per_min', 'avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_test.groupby('hero_id')[i].mean()
    data_test['gp_h{}_mean'.format(i)] = data_test['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_test.groupby('hero_id')[i].median()
    data_test['gp_h{}_median'.format(i)] = data_test['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    mn_1 =data_test.groupby('hero_id')[i].max()
    data_test['gp_h{}_max'.format(i)] = data_test['hero_id'].map(mn_1)
    
for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_test['delta_{}_mean'.format(i)] = data_test['gp_h{}_mean'.format(i)] - data_test[i]
    
for i in ['gold_per_min','xp_per_min', 'avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_test['delta_{}_max'.format(i)] = data_test['gp_h{}_max'.format(i)] - data_test[i]

for i in ['gold_per_min','xp_per_min','avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']:
    data_test['delta_{}_median'.format(i)] = data_test['gp_h{}_median'.format(i)] - data_test[i]

In [5]:
data_train = data_train.drop(['skilled','player_team', 'winner_team','Unnamed: 0' ,'Unnamed: 0.1','Unnamed: 0.1.1', 'Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1','Unnamed: 0.1.1.1.1.1.1.1'], axis=1)
data_test = data_test.drop(['Unnamed: 0','player_team', 'winner_team', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1','Unnamed: 0.1.1.1.1.1.1.1'], axis=1)

In [7]:
X = data_train
x_test = data_test
y = target

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
my_model = RandomForestClassifier(random_state=0).fit(X_train, y_train)

# Bagging

acc
------
0.7086595060379878<br>
0.7285939680805874<br>
0.7359366466411796<br>

In [30]:
bags = 30
seed = 1 

bagged_prediction = np.zeros(X_test.shape[0])

for n in tqdm_notebook(range(0,bags)):
    my_model.set_params(random_state= seed + n)
    my_model.fit(X_train, y_train)
    preds = my_model.predict(X_test)
    bagged_prediction += preds

In [31]:
bagged_prediction /= bags

In [32]:
(bagged_prediction > 0.5).astype('int')

array([0, 1, 0, ..., 0, 0, 0])

In [33]:
accuracy_score(y_test, (bagged_prediction > 0.5).astype('int'))

0.7371806541659082

# Stacking

In [91]:
rg0 = Ridge(alpha=0.01)
rg1 = Ridge(alpha=1.1)
clf_1 = catboost.CatBoostClassifier(max_depth=5, )    
clf_2 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005, objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, seed=27)
clf_3 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10
clf_4  = xgboost.XGBClassifier(max_depth=12, learning_rate=0.070,n_estimators=1500, min_child_weight=11, gamma=1.000, subsample=0.6, colsample_bytree=0.3, reg_alpha=3.5,objective='binary:logitraw', scale_pos_weight=2.2)  

In [92]:
for clf in tqdm_notebook([rg0, rg1, clf_1, clf_2, clf_3, clf_4]):
    clf.fit(X_train, y_train)
    print(accuracy_score(y_test, (clf.predict(X_test) > 0.5).astype('int')))

0.7987624158923422
0.7987423902595322
Learning rate set to 0.050673
0:	learn: 0.6829839	total: 401ms	remaining: 6m 40s
1:	learn: 0.6739784	total: 731ms	remaining: 6m 4s
2:	learn: 0.6656215	total: 1.03s	remaining: 5m 43s
3:	learn: 0.6581490	total: 1.34s	remaining: 5m 34s
4:	learn: 0.6515410	total: 1.67s	remaining: 5m 32s
5:	learn: 0.6457190	total: 1.99s	remaining: 5m 30s
6:	learn: 0.6395836	total: 2.35s	remaining: 5m 32s
7:	learn: 0.6341399	total: 2.69s	remaining: 5m 33s
8:	learn: 0.6290575	total: 3.04s	remaining: 5m 34s
9:	learn: 0.6246890	total: 3.33s	remaining: 5m 29s
10:	learn: 0.6202665	total: 3.63s	remaining: 5m 26s
11:	learn: 0.6163072	total: 3.97s	remaining: 5m 27s
12:	learn: 0.6128011	total: 4.32s	remaining: 5m 27s
13:	learn: 0.6092726	total: 4.66s	remaining: 5m 28s
14:	learn: 0.6053725	total: 5s	remaining: 5m 28s
15:	learn: 0.6023989	total: 5.36s	remaining: 5m 29s
16:	learn: 0.5997867	total: 5.67s	remaining: 5m 28s
17:	learn: 0.5971088	total: 6.03s	remaining: 5m 28s
18:	learn:

157:	learn: 0.4952424	total: 55.2s	remaining: 4m 54s
158:	learn: 0.4949497	total: 55.5s	remaining: 4m 53s
159:	learn: 0.4946351	total: 55.8s	remaining: 4m 52s
160:	learn: 0.4943965	total: 56.2s	remaining: 4m 52s
161:	learn: 0.4941176	total: 56.6s	remaining: 4m 52s
162:	learn: 0.4937864	total: 56.9s	remaining: 4m 52s
163:	learn: 0.4935306	total: 57.3s	remaining: 4m 51s
164:	learn: 0.4932958	total: 57.6s	remaining: 4m 51s
165:	learn: 0.4930366	total: 58s	remaining: 4m 51s
166:	learn: 0.4927213	total: 58.3s	remaining: 4m 50s
167:	learn: 0.4921820	total: 58.6s	remaining: 4m 50s
168:	learn: 0.4918837	total: 58.9s	remaining: 4m 49s
169:	learn: 0.4916167	total: 59.3s	remaining: 4m 49s
170:	learn: 0.4913708	total: 59.6s	remaining: 4m 49s
171:	learn: 0.4911023	total: 60s	remaining: 4m 48s
172:	learn: 0.4909170	total: 1m	remaining: 4m 48s
173:	learn: 0.4906170	total: 1m	remaining: 4m 48s
174:	learn: 0.4902938	total: 1m 1s	remaining: 4m 47s
175:	learn: 0.4898213	total: 1m 1s	remaining: 4m 47s
176

311:	learn: 0.4595177	total: 1m 49s	remaining: 4m 2s
312:	learn: 0.4593989	total: 1m 50s	remaining: 4m 2s
313:	learn: 0.4591515	total: 1m 50s	remaining: 4m 1s
314:	learn: 0.4590113	total: 1m 50s	remaining: 4m 1s
315:	learn: 0.4588858	total: 1m 51s	remaining: 4m 1s
316:	learn: 0.4587498	total: 1m 51s	remaining: 4m
317:	learn: 0.4586252	total: 1m 52s	remaining: 4m
318:	learn: 0.4584277	total: 1m 52s	remaining: 3m 59s
319:	learn: 0.4582320	total: 1m 52s	remaining: 3m 59s
320:	learn: 0.4580097	total: 1m 53s	remaining: 3m 59s
321:	learn: 0.4577858	total: 1m 53s	remaining: 3m 58s
322:	learn: 0.4575819	total: 1m 53s	remaining: 3m 58s
323:	learn: 0.4573919	total: 1m 54s	remaining: 3m 58s
324:	learn: 0.4572056	total: 1m 54s	remaining: 3m 57s
325:	learn: 0.4570382	total: 1m 54s	remaining: 3m 57s
326:	learn: 0.4568980	total: 1m 55s	remaining: 3m 57s
327:	learn: 0.4567556	total: 1m 55s	remaining: 3m 56s
328:	learn: 0.4566578	total: 1m 55s	remaining: 3m 56s
329:	learn: 0.4564960	total: 1m 56s	remai

464:	learn: 0.4376240	total: 2m 44s	remaining: 3m 9s
465:	learn: 0.4375078	total: 2m 44s	remaining: 3m 8s
466:	learn: 0.4373994	total: 2m 45s	remaining: 3m 8s
467:	learn: 0.4372882	total: 2m 45s	remaining: 3m 8s
468:	learn: 0.4371901	total: 2m 45s	remaining: 3m 7s
469:	learn: 0.4370876	total: 2m 46s	remaining: 3m 7s
470:	learn: 0.4369795	total: 2m 46s	remaining: 3m 7s
471:	learn: 0.4367695	total: 2m 46s	remaining: 3m 6s
472:	learn: 0.4366217	total: 2m 47s	remaining: 3m 6s
473:	learn: 0.4365141	total: 2m 47s	remaining: 3m 5s
474:	learn: 0.4364195	total: 2m 47s	remaining: 3m 5s
475:	learn: 0.4363191	total: 2m 48s	remaining: 3m 5s
476:	learn: 0.4361938	total: 2m 48s	remaining: 3m 4s
477:	learn: 0.4360838	total: 2m 49s	remaining: 3m 4s
478:	learn: 0.4359332	total: 2m 49s	remaining: 3m 4s
479:	learn: 0.4358188	total: 2m 49s	remaining: 3m 3s
480:	learn: 0.4356451	total: 2m 50s	remaining: 3m 3s
481:	learn: 0.4354821	total: 2m 50s	remaining: 3m 3s
482:	learn: 0.4353913	total: 2m 50s	remaining:

618:	learn: 0.4205806	total: 3m 39s	remaining: 2m 14s
619:	learn: 0.4205202	total: 3m 39s	remaining: 2m 14s
620:	learn: 0.4204060	total: 3m 39s	remaining: 2m 14s
621:	learn: 0.4202914	total: 3m 40s	remaining: 2m 13s
622:	learn: 0.4201401	total: 3m 40s	remaining: 2m 13s
623:	learn: 0.4199949	total: 3m 40s	remaining: 2m 13s
624:	learn: 0.4199290	total: 3m 41s	remaining: 2m 12s
625:	learn: 0.4198544	total: 3m 41s	remaining: 2m 12s
626:	learn: 0.4197907	total: 3m 41s	remaining: 2m 12s
627:	learn: 0.4196950	total: 3m 42s	remaining: 2m 11s
628:	learn: 0.4195478	total: 3m 42s	remaining: 2m 11s
629:	learn: 0.4194148	total: 3m 42s	remaining: 2m 10s
630:	learn: 0.4193302	total: 3m 43s	remaining: 2m 10s
631:	learn: 0.4192629	total: 3m 43s	remaining: 2m 10s
632:	learn: 0.4191878	total: 3m 44s	remaining: 2m 9s
633:	learn: 0.4191029	total: 3m 44s	remaining: 2m 9s
634:	learn: 0.4190240	total: 3m 44s	remaining: 2m 9s
635:	learn: 0.4189411	total: 3m 45s	remaining: 2m 8s
636:	learn: 0.4188342	total: 3m 

772:	learn: 0.4067700	total: 4m 33s	remaining: 1m 20s
773:	learn: 0.4066859	total: 4m 33s	remaining: 1m 19s
774:	learn: 0.4066258	total: 4m 33s	remaining: 1m 19s
775:	learn: 0.4065644	total: 4m 34s	remaining: 1m 19s
776:	learn: 0.4064665	total: 4m 34s	remaining: 1m 18s
777:	learn: 0.4064084	total: 4m 34s	remaining: 1m 18s
778:	learn: 0.4063510	total: 4m 35s	remaining: 1m 18s
779:	learn: 0.4062883	total: 4m 35s	remaining: 1m 17s
780:	learn: 0.4062140	total: 4m 35s	remaining: 1m 17s
781:	learn: 0.4061452	total: 4m 36s	remaining: 1m 17s
782:	learn: 0.4060731	total: 4m 36s	remaining: 1m 16s
783:	learn: 0.4060079	total: 4m 37s	remaining: 1m 16s
784:	learn: 0.4059377	total: 4m 37s	remaining: 1m 15s
785:	learn: 0.4058254	total: 4m 37s	remaining: 1m 15s
786:	learn: 0.4057300	total: 4m 38s	remaining: 1m 15s
787:	learn: 0.4056260	total: 4m 38s	remaining: 1m 14s
788:	learn: 0.4055501	total: 4m 38s	remaining: 1m 14s
789:	learn: 0.4054812	total: 4m 39s	remaining: 1m 14s
790:	learn: 0.4054011	total:

928:	learn: 0.3950763	total: 5m 28s	remaining: 25.1s
929:	learn: 0.3950012	total: 5m 28s	remaining: 24.7s
930:	learn: 0.3949467	total: 5m 29s	remaining: 24.4s
931:	learn: 0.3948791	total: 5m 29s	remaining: 24s
932:	learn: 0.3948263	total: 5m 29s	remaining: 23.7s
933:	learn: 0.3947828	total: 5m 30s	remaining: 23.3s
934:	learn: 0.3947143	total: 5m 30s	remaining: 23s
935:	learn: 0.3946374	total: 5m 30s	remaining: 22.6s
936:	learn: 0.3945909	total: 5m 31s	remaining: 22.3s
937:	learn: 0.3945013	total: 5m 31s	remaining: 21.9s
938:	learn: 0.3944174	total: 5m 31s	remaining: 21.6s
939:	learn: 0.3943378	total: 5m 32s	remaining: 21.2s
940:	learn: 0.3942711	total: 5m 32s	remaining: 20.8s
941:	learn: 0.3941960	total: 5m 32s	remaining: 20.5s
942:	learn: 0.3941212	total: 5m 33s	remaining: 20.1s
943:	learn: 0.3940647	total: 5m 33s	remaining: 19.8s
944:	learn: 0.3939656	total: 5m 33s	remaining: 19.4s
945:	learn: 0.3939186	total: 5m 34s	remaining: 19.1s
946:	learn: 0.3938545	total: 5m 34s	remaining: 18.

In [93]:
preds1 = rg0.predict(X_test)
preds2 = rg1.predict(X_test)
preds3 = clf_1.predict(X_test)
preds4 = clf_2.predict(X_test)
preds5 = clf_3.predict(X_test)
preds6 = clf_4.predict(X_test)

In [94]:
test_preds1 = rg0.predict(x_test)
test_preds2 = rg1.predict(x_test)
test_preds3 = clf_1.predict(x_test)
test_preds4 = clf_2.predict(x_test)
test_preds5 = clf_3.predict(x_test)
test_preds6 = clf_4.predict(x_test)

In [95]:
stacked_preds = np.column_stack((preds1, preds2, preds3, preds4,preds5, preds6 ))
stacked_test_preds = np.column_stack((test_preds1, test_preds2, test_preds3, test_preds4, test_preds5, test_preds6))

In [96]:
meta_model = Ridge(0.001)

In [97]:
meta_model.fit(stacked_preds, y_test)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [98]:
pred = meta_model.predict(stacked_test_preds)

## Sub

In [99]:
pred = (pred > 0.5).astype('int')

In [100]:
import time
iss = 0

def make_submit(pred):
    global iss
    submit = pd.DataFrame()
    submit['id'] = data_test['id']
    submit['skilled'] = pred
    iss += 1
    submit.to_csv('submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss), index=False)
    print('last: ', 'submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss))

In [101]:
make_submit(pred)

last:  submissions/submission31-1.csv


In [83]:
rg0 = Ridge(alpha=20, )
rg0.fit(X_train, y_train)
print(accuracy_score(y_test, (rg0.predict(X_test) > 0.5).astype('int')))

0.7986622877282922


In [89]:
las = Lasso(alpha=0.1)
las.fit(X_train, y_train)
print(accuracy_score(y_test, (las.predict(X_test) > 0.5).astype('int')))

0.7580102531239987
